# experiment for paper

In [12]:
import os
import json
from collections import defaultdict
import subprocess


# Build Tool Database

In [13]:
folders = os.listdir("../extractor/apidocs")
parameter_dict = defaultdict(set)
def add_to_dict(d: defaultdict, prev_path: str, object, api_doc: str):
    '''encode a json tree into a dictionary with root to leaf path. sets have max length of 10'''
    if isinstance(object, dict):
        for key, value in object.items():
            add_to_dict(d, prev_path + "[" + key + "]", value, api_doc)
    elif isinstance(object, list):
        for item in object:
            add_to_dict(d, prev_path, item, api_doc)
    else:
        if object:
            if len(d[prev_path]) < 10:
                d[prev_path].add((api_doc, object))
def build_dict(d, object, api_doc):
    add_to_dict(d, '', object, api_doc)
for folder in folders:
    path = "../extractor/apidocs/" + folder + "/" + folder + ".txt"
    try:
        json_file = json.load(open(path))
    except:
        continue
    for endpoint in json_file['endpoints']:
        for param in endpoint['required_parameters']:
            name = param['name']
            example = None if not param['example'] else param['example']
            add_to_dict(parameter_dict, "["+name+"]", example, folder)
            
        if endpoint['optional_parameters']:
            for param in endpoint['optional_parameters']:
                name = param['name']
                example = None if not param['example'] else param['example']
                add_to_dict(parameter_dict, "["+name+"]", example, folder)
parameter_dict


defaultdict(set,
            {'[sequence]': {('glycam',
               'DManpa1-2DManpa1-3DManpa1-6[DManpa1-3]DManpb1-4DGlcpNAcb1-4DGlcpNAcb1-OH'),
              ('glycam-new', 'DManpa1-6DManpa1-OH'),
              ('glycam2', 'DManpa1-6DManpa1-OH')},
             '[buildOptions][conformers]': {('glycam-new', '1ogg'),
              ('glycam2', 64)},
             '[pUUID]': {('glycam-new',
               '3c368bf2-ad73-43f3-a18d-d7d2dc11cf28'),
              ('glycam2', '3c368bf2-ad73-43f3-a18d-d7d2dc11cf28')},
             '[conformerID]': {('glycam-new', '1ogg'), ('glycam2', '1ogg')},
             '[category-name]': {('example', 'food-and-drink')},
             '[group-name]': {('example', 'animal-bird')}})

In [14]:
folders = os.listdir("../extractor/apidocs")
description_to_param_dict = {} # map description to parameter
param_to_description_dict = {} # map parameter to description
for folder in folders:
    path = "../extractor/apidocs/" + folder + "/" + folder + ".txt"
    try:
        json_file = json.load(open(path))
    except:
        continue
    for endpoint in json_file['endpoints']:
        for param in endpoint['required_parameters']:
            name = param['name']
            description = param['description']
            description_to_param_dict[description] = name
            param_to_description_dict[name] = description
        if endpoint['optional_parameters']:
            for param in endpoint['optional_parameters']:
                name = param['name']
                description = param['description']
                description_to_param_dict[description] = name
                param_to_description_dict[name] = description

description_to_param_dict

{'The sequence to be evaluated.': 'sequence',
 'The sequence for which to build the 3D structure.': 'sequence',
 'Options for building the 3D structure, such as number of conformers.': 'buildOptions',
 'The project UUID to check the status for.': 'pUUID',
 'The conformer ID to check the status for.': 'conformerID',
 'The project UUID for the structure to download.': 'pUUID',
 'The name of the category to filter emojis by.': 'category-name',
 'The name of the group to filter emojis by.': 'group-name',
 'Options for building the structure, such as specific conformers.': 'buildOptions',
 'The ID of the conformer to check the status for.': 'conformerID',
 'The sequence of the sugar expressed in condensed glycam notation.': 'sequence'}

# Get API response

In [4]:
# no need to run again
apidocs_dir = "../extractor/apidocs"
count = 0
evaluation_result={}
folders = os.listdir(apidocs_dir)
for folder in folders:
    files = os.listdir(os.path.join(apidocs_dir, folder))
    failed_endpoints = []
    api_result={}
    files = [x for x in files if x.endswith(".py")]
    for file_name in files:
        if file_name.endswith(".py"):
            count += 1
            file_path = os.path.join(apidocs_dir, folder, file_name)
            try:
                result = subprocess.run(
                    ["python", file_path], capture_output=True, text=True, check=True
                )
                if result.stdout:
                    output = result.stdout
                    output_json = json.loads(output)
                    # save in file
                    with open(file_path[:-3] + '_response.json', "w") as f:
                        json.dump(output_json, f)
                else:
                    pass
            except subprocess.CalledProcessError as e:
                pass
            print(f"Tested: {file_name}")


Tested: poll_project_status_GET.py
Tested: poll_specific_build_status_GET.py
Tested: download_structure_GET.py
Tested: evaluate_sequence_POST.py
Tested: build_3d_structure_POST.py
Tested: get_emojis_by_group_GET.py
Tested: get_random_emoji_GET.py
Tested: get_emojis_by_category_GET.py
Tested: get_all_emojis_GET.py
Tested: poll_project_status_GET.py
Tested: poll_specific_build_status_GET.py
Tested: download_structure_GET.py
Tested: evaluate_sequence_POST.py
Tested: build_3d_structure_POST.py
Tested: build_structure_via_url_GET.py


In [5]:
apidocs_dir = "../extractor/apidocs"
count = 0
evaluation_result={}
folders = os.listdir(apidocs_dir)
response_dict = defaultdict(set)
for folder in folders:
    files = os.listdir(os.path.join(apidocs_dir, folder))
    failed_endpoints = []
    api_result={}
    files = [x for x in files if x.endswith("_response.json")]
    for file_name in files:
        file_path = os.path.join(apidocs_dir, folder, file_name)
        response = json.load(open(file_path))
        if response['status_code'] == 200:
            response_json = response['json']
            build_dict(response_dict, response_json, folder)
response_dict



defaultdict(set, {})

In [9]:
len(response_dict)

0

# Calculate embedding, build parameter knowledge base

In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")
param_keys = list(parameter_dict.keys())
param_keys_emb = model.encode(param_keys, convert_to_tensor=True)
response_keys = list(response_dict.keys())
response_keys_emb = model.encode(response_keys, convert_to_tensor=True)
description_keys = list(description_to_param_dict.keys())
description_keys_emb = model.encode(description_keys, convert_to_tensor=True)


/Users/jianhaonan/Desktop/research/APIAgent/.venv/lib/python3.13/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
query = 'iupacextended'
query_emb = model.encode(query, convert_to_tensor=True) 
hits_param = util.semantic_search(query_emb, param_keys_emb)
hits_response = util.semantic_search(query_emb, response_keys_emb)
query_description = param_to_description_dict[query]
query_description_emb = model.encode(query_description, convert_to_tensor=True)
hits_description = util.semantic_search(query_description_emb, description_keys_emb)

KeyError: 'iupacextended'

In [ ]:
h = hits_param[0]
for hit in h:
    print(f"Param: {param_keys[hit['corpus_id']]} Example: {parameter_dict[param_keys[hit['corpus_id']]]}")
    print(f"Similarity: {hit['score']}")
    print("")

print("==================================")
h = hits_response[0]
for hit in h:
    print(f"Request: {response_keys[hit['corpus_id']]} Example: {response_dict[response_keys[hit['corpus_id']]]}")
    print(f"Similarity: {hit['score']}")
    print("")

print("==================================")
h = hits_description[0]
for hit in h:
    print(f"Description: {description_keys[hit['corpus_id']]} Example: {parameter_dict['['+description_to_param_dict[description_keys[hit['corpus_id']]]+']']}")
    print(f"Similarity: {hit['score']}")
    print("")

# load generated tools

In [7]:
import sys
def load_and_import(api_name, function_name):
    folder_path = os.path.join("..","extractor", "apidocs", api_name)
    sys.path.insert(0, folder_path)
    module = __import__(function_name)
    sys.path.pop(0)  # Clean up after import
    return module

module = load_and_import("glycanformatconverter", "convert_glycoct_to_wurcs_GET")

In [ ]:
f = getattr(module, "convert_glycoct_to_wurcs")
response = f(glycoct='''RES\n1b:x-dglc-HEX-1:5\n2s:n-acetyl\n3b:b-dglc-HEX-1:5\n4s:n-acetyl\n5b:b-dman-HEX-1:5\n6b:a-dman-HEX-1:5\n7b:a-dman-HEX-1:5\nLIN\n1:1d(2+1)2n\n2:1o(4+1)3d\n3:3d(2+1)4n\n4:3o(4+1)5d\n5:5o(3+1)6d\n6:5o(6+1)7d''')


# query KB and get parameter value candidates

In [34]:
def get_function(api_name, endpoint_name):
    function_name = endpoint_name.replace("_GET", "").replace("_POST", "")
    module = load_and_import(api_name, endpoint_name)
    f = getattr(module, function_name)
    return f

import random
def get_test_examples(query, api_doc_name, max_param_examples=5, max_response_examples=5, max_examples_per_hit=1, similarity_threshold=0.5, query_description=None):
    if not query_description:
        try:
            query_description = param_to_description_dict[query]
        except:
            query_description = None
    query_emb = model.encode(query, convert_to_tensor=True) 
    hits_param = util.semantic_search(query_emb, param_keys_emb)
    hits_response = util.semantic_search(query_emb, response_keys_emb)
    if query_description:
        query_description_emb = model.encode(query_description, convert_to_tensor=True)
        hits_description = util.semantic_search(query_description_emb, description_keys_emb)
    hit_list = hits_param[0]
    all_examples = set()
    count = 0
    for hit in hit_list:
        if hit['score'] < similarity_threshold or count > max_param_examples:
            break
        param_key = param_keys[hit['corpus_id']]
        examples_set = parameter_dict[param_key]
        filted_examples = [t[1] for t in examples_set if t[0]!= api_doc_name]
        # print([(t[0],t[1]) for t in examples_set if t[0]!= api_doc_name])
        examples = random.sample(filted_examples, min(len(filted_examples), max_examples_per_hit))
        sidx = param_key.rfind('[')+1
        test_param = param_key[sidx:-1]
        # print(test_param, examples)
        pairs = [(test_param, example) for example in examples]
        all_examples.update(pairs)
        # print(all_examples)
        count += len(examples)
    hit_list = hits_response[0]
    count = 0
    for hit in hit_list:
        if hit['score'] < similarity_threshold or count > max_response_examples:
            break
        response_key = response_keys[hit['corpus_id']]
        examples_set = response_dict[response_key]
        examples_set_copy = examples_set.copy()
        filted_examples = [t[1] for t in examples_set if t[0]!= api_doc_name]
        # print([(t[0],t[1]) for t in examples_set if t[0]!= api_doc_name])
        examples = random.sample(filted_examples, min(len(filted_examples), max_examples_per_hit))
        sidx = response_key.rfind('[')+1
        test_param = response_key[sidx:-1]
        # print(test_param, examples)
        pairs = [(test_param, example) for example in examples]
        all_examples.update(pairs)
        # print(all_examples)
        count += len(examples)
    if query_description:
        hit_list = hits_description[0]
        count = 0
        for hit in hit_list:
            if hit['score'] < similarity_threshold or count > max_param_examples:
                break
            description_key = description_keys[hit['corpus_id']]
            param_key = description_to_param_dict[description_key]
            examples_set = parameter_dict['['+param_key+']']
            filted_examples = [t[1] for t in examples_set if t[0]!= api_doc_name]
            # print([(t[0],t[1]) for t in examples_set if t[0]!= api_doc_name])
            examples = random.sample(filted_examples, min(len(filted_examples), max_examples_per_hit))
            # print(param_key, examples)
            pairs = [(param_key, example) for example in examples]
            all_examples.update(pairs)
            # print(all_examples)
            count += len(examples)
    return all_examples

    

In [ ]:
get_test_examples("iupacextended", "glycanformatconverter", max_param_examples=5, max_response_examples=5, max_examples_per_hit=1, similarity_threshold=0.5)

In [36]:
import inspect
import time
import itertools
def get_parameter_names(func):
    signature = inspect.signature(func)
    return [param.name for param in signature.parameters.values()]
def run_experiment(MAX_SAMPLES=100):
    apidocs_dir = "../extractor/apidocs"
    folders = os.listdir(apidocs_dir)
    for folder in folders:
        files = os.listdir(os.path.join(apidocs_dir, folder))
        files = [x for x in files if x.endswith(".py")]
        api_extracted_json = {}
        with open(os.path.join(apidocs_dir, folder, folder + ".txt")) as f:
            api_extracted_json = json.load(f)
        extracted_endpoints = None
        try:
            extracted_endpoints = api_extracted_json['endpoints']
        except:
            print("No extracted information for api: ", folder)
            extracted_endpoints = None
            continue
        for idx, file_name in enumerate(files):
            endpoint_name = file_name.replace(".py", "")
            record_file_name = endpoint_name + "_example_test.json"
            if os.path.exists(os.path.join(apidocs_dir, folder, record_file_name)):
                endpoint_result = json.load(open(os.path.join(apidocs_dir, folder, record_file_name)))
            else:
                endpoint_result = {"endpoint": endpoint_name, "tests": [], "extracted_parameters": {}, "validated_parameters": {}}
                print(f"Testing {folder}.{endpoint_name}")
                if file_name == "__init__.py":
                    continue
                
            if extracted_endpoints:
                # find the ground truth parameters
                pass
                # endpoint_result['extracted_parameters'] = {}
                # current_endpoint = None
                # for endpoint in extracted_endpoints:
                #     current_endpoint_name = endpoint['name'].replace(' ','_').lower() + '_' + endpoint['method']
                #     if current_endpoint_name == endpoint_name:
                #         current_endpoint = endpoint
                #         print(f"Found endpoint {endpoint_name} in extracted information")
                #         break
                # if not current_endpoint:
                #     print(f"Endpoint {endpoint_name} not found in extracted information")
                #     continue
                # for req_param in current_endpoint['required_parameters']:
                #     endpoint_result['extracted_parameters'][req_param['name']] = {'description': req_param['description'], 'example': req_param['example']}
                # if current_endpoint['optional_parameters']:
                #     for opt_param in current_endpoint['optional_parameters']:
                #         endpoint_result['extracted_parameters'][opt_param['name']] = {'description': opt_param['description'], 'example': opt_param['example']}
            f = get_function(folder, endpoint_name)
            param_names = get_parameter_names(f)
            # simulate the situation that no example is provided
            # for a single parameter function, we can try a few test cases.
            if len(param_names) == 1:
                endpoint_result['tests'] = []
                curr_param = param_names[0]
                examples = get_test_examples(curr_param, folder, max_param_examples=5, max_response_examples=5, max_examples_per_hit=1, similarity_threshold=0.5)
                for test_param, example in examples:
                    try:
                        response = f(example)
                    except:
                        print(f'Error occured when calling {folder}.{endpoint_name} with {curr_param}={example}')
                        continue
                    response_json =""
                    try:
                        response_json = response.json()
                    except:
                        pass
                    example_result = {'gt_param': curr_param,'test_param': test_param, 'candidate': example, 'status_code': response.status_code, 'json': response_json, 'text': response.text}
                    endpoint_result['tests'].append(example_result)
                    time.sleep(0.1)
                pass 
            else:
                # use a rough estimation, record the ranking of the ground truth parameter(?)
                endpoint_result['tests'] = []
                example_list = []
                for param in param_names:
                    examples = get_test_examples(param, folder, max_param_examples=5, max_response_examples=5, max_examples_per_hit=1, similarity_threshold=0.5)
                    example_list.append(examples)
                candidate_combinations = list(itertools.product(*example_list))
                samples = random.sample(candidate_combinations, min(MAX_SAMPLES, len(candidate_combinations)))
                print(f"Testing {folder}.{endpoint_name} with {len(samples)} samples")
                for sample in samples:
                    try:
                        input_dict = {param_names[i]: sample[i][1] for i in range(len(param_names))}
                        response = f(**input_dict)
                    except:
                        print(f'Error occured when calling {folder}.{endpoint_name} with {input_dict}')
                        continue
                    response_json =""
                    try:
                        response_json = response.json()
                    except:
                        pass
                    example_result = {'gt_param': param_names,'test_param': [t[0] for t in sample], 'candidate': [t[1] for t in sample], 'status_code': response.status_code, 'json': response_json, 'text': response.text}
                    endpoint_result['tests'].append(example_result)
                    time.sleep(0.1)
                pass
            # save endpoint_result
            save_path = os.path.join(apidocs_dir, folder, endpoint_name + "_example_test.json")
            with open(save_path, "w") as f:
                json.dump(endpoint_result, f)



In [ ]:
run_experiment()

# let gpt evaluate the tested examples
1. check if status code is 200
2. chunk the response
3. sent response to gpt, let it check if the API returns an actual response rather than an error information

In [38]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
tagging_prompt = ChatPromptTemplate.from_template(
    """
Decide if the following API response is an information or an error message.

API Description:
{description}
API Response:
{response}
"""
)
class Classification(BaseModel):
    response_type: str= Field(..., enum=['information', 'error'])
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

def gpt_evaluate(API_description:str, API_response: str):
    '''let gpt to decide if the API response is a piece of information or an error message'''
    prompt = tagging_prompt.invoke({"description": API_description, "response": API_response})
    gpt_response = llm.invoke(prompt)
    return gpt_response.response_type

In [ ]:
import os
import json
apidocs_dir = "../extractor/apidocs"
folders = os.listdir(apidocs_dir)
count_single_param_files = 0
count_multi_param_files = 0
count_validated_files = 0
count_validated_single_param_files = 0
count_validated_multi_param_files = 0
count_find_by_scemantic = 0
count_find_by_scemantic_single = 0
count_find_by_scemantic_multi = 0
scemantic_case = []
for folder in folders:
    files = os.listdir(os.path.join(apidocs_dir, folder))
    files = [x for x in files if x.endswith("_example_test.json")]
    for file_name in files:
        file_path = os.path.join(apidocs_dir, folder, file_name)
        # if json file is larger than 10MB, skip it
        if os.path.getsize(file_path) > 100*1024*1024:
            continue
        endpoint_result = json.load(open(file_path))
        endpoint_result['validated_parameters'] = {}
        print(f"Validating {folder}.{endpoint_result['endpoint']}")
        saved_error_responses = set() # API responses usually have similar error messages, save them to avoid repeated evaluation
        if endpoint_result['tests']:
            if type(endpoint_result['tests'][0]['gt_param']) == str:
                count_single_param_files += 1
            else:
                count_multi_param_files += 1
        for test in endpoint_result['tests']:
            status_code = test['status_code']
            if status_code == 200:
                response_text = test['text']
                # truncate the response text to avoid the max token limit of gpt
                response_text = response_text[:500] if len(response_text) > 500 else response_text
                if response_text in saved_error_responses:
                    continue
                else:
                    # if type(test['gt_param']) == str:
                    #     continue
                    gpt_response = gpt_evaluate(endpoint_result['endpoint'],response_text)
                    if gpt_response == 'error':
                        print(f"Error response detected: {response_text}")
                        saved_error_responses.add(response_text)
                    else:
                        print(f"Information response detected: {response_text}")
                        count_validated_files += 1
                        if type(test['gt_param']) == str:
                            endpoint_result['validated_parameters'][test['gt_param']] = test['candidate']
                            count_validated_single_param_files += 1
                            if not test['test_param'] in endpoint_result['extracted_parameters']:
                                count_find_by_scemantic += 1
                                count_find_by_scemantic_single += 1
                                scemantic_case.append((folder, endpoint_result['endpoint'], test['test_param'], test['gt_param']))
                        else:
                            count_validated_multi_param_files += 1
                            for i, gt_param in enumerate(test['gt_param']):
                                endpoint_result['validated_parameters'][gt_param] = test['candidate'][i]
                                if not test['test_param'][i] in endpoint_result['extracted_parameters']:
                                    count_find_by_scemantic += 1
                                    count_find_by_scemantic_multi += 1
                                    scemantic_case.append((folder, endpoint_result['endpoint'], test['test_param'][i], gt_param))
                        break
        with open(file_path, "w") as f:
            json.dump(endpoint_result, f)
print(f"Validated {count_validated_files} parameters in {count_multi_param_files+count_single_param_files} files. {count_find_by_scemantic} parameters are found by semantic search.") 
print(f"Validated {count_validated_single_param_files} single parameters in {count_single_param_files} files. {count_find_by_scemantic_single} single parameters are found by semantic search.")
print(f"Validated {count_validated_multi_param_files} multi parameters in {count_multi_param_files} files. {count_find_by_scemantic_multi} multi parameters are found by semantic search.")
print(scemantic_case)

            

In [ ]:
import ast
import datetime
import shutil
from typing import Dict, List, Any, Set, Optional, Union

# Cell 39: Process Validated Parameters
def process_validated_parameters(apidocs_dir: str) -> Dict:
    """
    Process test results to extract validated parameters.
    
    Args:
        apidocs_dir: Directory containing API documentation and test results
        
    Returns:
        Dictionary mapping API endpoints to their validated parameters
    """
    validated_params = {}
    folders = os.listdir(apidocs_dir)
    
    for folder in folders:
        folder_path = os.path.join(apidocs_dir, folder)
        if not os.path.isdir(folder_path):
            continue
            
        files = os.listdir(folder_path)
        test_files = [x for x in files if x.endswith("_example_test.json")]
        
        for file_name in test_files:
            file_path = os.path.join(folder_path, file_name)
            try:
                with open(file_path, 'r') as f:
                    test_data = json.load(f)
                    if 'validated_parameters' in test_data and test_data['validated_parameters']:
                        validated_params[f"{folder}.{test_data['endpoint']}"] = test_data['validated_parameters']
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    
    return validated_params

In [ ]:
# Cell 40: Infer Parameter Types
def infer_parameter_type(value: Any) -> str:
    """
    Infer the type of a parameter value.
    
    Args:
        value: The parameter value
        
    Returns:
        A string representing the inferred type
    """
    if value is None:
        return "None"
    elif isinstance(value, bool):
        return "bool"
    elif isinstance(value, int):
        return "int"
    elif isinstance(value, float):
        return "float"
    elif isinstance(value, str):
        # Check if it's a date string
        if len(value) == 10 and value.count('-') == 2:
            try:
                datetime.datetime.strptime(value, '%Y-%m-%d')
                return "date"
            except ValueError:
                pass
        # Check if it's a JSON string
        if (value.startswith('{') and value.endswith('}')) or (value.startswith('[') and value.endswith(']')):
            try:
                json.loads(value)
                return "json"
            except json.JSONDecodeError:
                pass
        return "str"
    elif isinstance(value, (list, tuple)):
        return "list"
    elif isinstance(value, dict):
        return "dict"
    else:
        return type(value).__name__

def analyze_parameter_types(validated_params: Dict) -> Dict:
    """
    Analyze and infer types for validated parameters.
    
    Args:
        validated_params: Dictionary of validated parameters
        
    Returns:
        Dictionary with parameter types added
    """
    typed_params = {}
    
    for tool_id, params in validated_params.items():
        typed_params[tool_id] = {}
        for param_name, param_value in params.items():
            param_type = infer_parameter_type(param_value)
            typed_params[tool_id][param_name] = {
                'value': param_value,
                'type': param_type
            }
    
    return typed_params

In [ ]:
# Cell 41: Update Tool Code
def update_tool_code(api_name: str, endpoint_name: str, params: Dict) -> bool:
    """
    Update tool code with validated parameters.
    
    Args:
        api_name: Name of the API
        endpoint_name: Name of the endpoint
        params: Dictionary of parameter names and their info (value and type)
        
    Returns:
        True if update was successful, False otherwise
    """
    tool_path = os.path.join("../extractor/apidocs", api_name, f"{endpoint_name}.py")
    
    if not os.path.exists(tool_path):
        print(f"Tool file not found: {tool_path}")
        return False
    
    try:
        # Create a backup of the original file
        backup_path = f"{tool_path}.bak"
        shutil.copy2(tool_path, backup_path)
        
        # Read the original code
        with open(tool_path, 'r') as f:
            code = f.read()
        
        # Parse the code
        tree = ast.parse(code)
        function_name = endpoint_name.replace("_GET", "").replace("_POST", "")
        
        # Find the main function in the file
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                # Get existing docstring
                docstring = ast.get_docstring(node)
                
                # Update docstring with validated parameters
                param_docs = "\n\nValidated Parameters:\n"
                for param_name, param_info in params.items():
                    param_value = param_info['value']
                    param_type = param_info['type']
                    param_docs += f"    {param_name} ({param_type}): {param_value}\n"
                
                if docstring:
                    # Remove existing "Validated Parameters" section if it exists
                    if "Validated Parameters:" in docstring:
                        parts = docstring.split("Validated Parameters:")
                        docstring = parts[0].rstrip()
                    
                    # Add new section
                    new_docstring = f"{docstring}{param_docs}"
                    
                    # Replace docstring in function
                    node.body[0] = ast.Expr(value=ast.Constant(value=new_docstring))
                else:
                    # Create a new docstring
                    new_docstring = f"Function for {endpoint_name}.{param_docs}"
                    node.body.insert(0, ast.Expr(value=ast.Constant(value=new_docstring)))
                
                # Update default parameter values
                param_values = {name: info['value'] for name, info in params.items()}
                for i, arg in enumerate(node.args.args):
                    if arg.arg in param_values:
                        # Check if we need to add or update a default
                        if i >= len(node.args.defaults):
                            # Add a new default
                            node.args.defaults.append(ast.Constant(value=param_values[arg.arg]))
                        else:
                            # Update existing default
                            node.args.defaults[i] = ast.Constant(value=param_values[arg.arg])
        
        # Write updated code
        updated_code = ast.unparse(tree)
        with open(tool_path, 'w') as f:
            f.write(updated_code)
        
        print(f"Successfully updated {tool_path}")
        return True
    
    except Exception as e:
        print(f"Error updating tool code for {api_name}.{endpoint_name}: {e}")
        # Restore from backup
        if os.path.exists(backup_path):
            shutil.copy2(backup_path, tool_path)
        return False

In [ ]:
# Cell 42: Create Parameter Knowledge Base
def create_parameter_kb(validated_params: Dict, output_path: str) -> None:
    """
    Create a knowledge base from validated parameters.
    
    Args:
        validated_params: Dictionary of validated parameters
        output_path: Path to save the knowledge base
    """
    kb = {
        'tools': {},
        'parameters': {},
        'metadata': {
            'created_at': datetime.datetime.now().isoformat(),
            'total_endpoints': len(validated_params),
            'total_parameters': sum(len(params) for params in validated_params.values())
        }
    }
    
    # Organize by tool
    for tool_id, params in validated_params.items():
        api_name, endpoint_name = tool_id.split('.')
        
        # Add to tools section
        if api_name not in kb['tools']:
            kb['tools'][api_name] = {}
        
        kb['tools'][api_name][endpoint_name] = params
        
        # Also organize by parameter for cross-tool lookup
        for param_name, param_info in params.items():
            if param_name not in kb['parameters']:
                kb['parameters'][param_name] = []
            
            kb['parameters'][param_name].append({
                'tool_id': tool_id,
                'value': param_info['value'],
                'type': param_info['type']
            })
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save to file
    with open(output_path, 'w') as f:
        json.dump(kb, f, indent=2)
    
    print(f"Knowledge base created at: {output_path}")

In [ ]:
# Cell 43: Test the Knowledge Base
def test_knowledge_base(kb_path: str, api_name: str = None, endpoint_name: str = None, param_name: str = None):
    """
    Test the knowledge base.
    
    Args:
        kb_path: Path to the knowledge base
        api_name: Optional API name to test
        endpoint_name: Optional endpoint name to test
        param_name: Optional parameter name to test
    """
    # Check if the knowledge base exists
    if not os.path.exists(kb_path):
        print(f"Knowledge base not found at {kb_path}")
        return
    
    # Load the knowledge base
    with open(kb_path, 'r') as f:
        kb = json.load(f)
    
    print(f"Knowledge Base Metadata:")
    print(f"  Created: {kb['metadata']['created_at']}")
    print(f"  Total Endpoints: {kb['metadata']['total_endpoints']}")
    print(f"  Total Parameters: {kb['metadata']['total_parameters']}")
    
    # Print APIs and endpoints
    print("\nAPIs and Endpoints:")
    for api, endpoints in kb['tools'].items():
        print(f"  {api}:")
        for endpoint in endpoints:
            print(f"    - {endpoint}")
    
    # Print parameters
    print("\nParameters:")
    param_count = len(kb['parameters'])
    print(f"  Total unique parameters: {param_count}")
    if param_count > 0:
        print("  Sample parameters:")
        for i, (param, examples) in enumerate(kb['parameters'].items()):
            if i >= 5:  # Show only 5 examples
                break
            print(f"    - {param}: {len(examples)} examples")
    
    # Test specific API/endpoint if provided
    if api_name and api_name in kb['tools']:
        print(f"\nTesting API: {api_name}")
        if endpoint_name and endpoint_name in kb['tools'][api_name]:
            print(f"  Endpoint: {endpoint_name}")
            params = kb['tools'][api_name][endpoint_name]
            print(f"  Parameters:")
            for param_name, param_info in params.items():
                print(f"    - {param_name}: {param_info['value']} ({param_info['type']})")
    
    # Test specific parameter if provided
    if param_name and param_name in kb['parameters']:
        print(f"\nTesting Parameter: {param_name}")
        examples = kb['parameters'][param_name]
        print(f"  {len(examples)} examples found:")
        for example in examples:
            print(f"    - {example['value']} ({example['type']}) from {example['tool_id']}")

In [ ]:
# Cell 44: Run Refinement Process
def run_refinement_process():
    """Run the complete tool refinement process."""
    apidocs_dir = "../extractor/apidocs"
    kb_path = "../utils/parameter_kb.json"
    
    print("🏭 Starting ToolFactory Refinement Process 🏭")
    print("\n1️⃣ Processing validated parameters...")
    validated_params = process_validated_parameters(apidocs_dir)
    print(f"   Found {len(validated_params)} tools with validated parameters")
    
    print("\n2️⃣ Analyzing parameter types...")
    typed_params = analyze_parameter_types(validated_params)
    print(f"   Analyzed types for {sum(len(params) for params in typed_params.values())} parameters")
    
    print("\n3️⃣ Updating tool code with validated parameters...")
    updated_count = 0
    for tool_id, params in typed_params.items():
        api_name, endpoint_name = tool_id.split('.')
        if update_tool_code(api_name, endpoint_name, params):
            updated_count += 1
    print(f"   Updated {updated_count} out of {len(typed_params)} tools")
    
    print("\n4️⃣ Creating parameter knowledge base...")
    create_parameter_kb(typed_params, kb_path)
    
    print("\n✅ Tool refinement process completed!")
    print(f"   - Knowledge base available at: {kb_path}")
    
    return typed_params, kb_path

# Uncomment to run the process
# typed_params, kb_path = run_refinement_process()

In [ ]:
# Cell 45: Main Execution
if __name__ == "__main__":
    # Run the refinement process
    typed_params, kb_path = run_refinement_process()
    
    # Test the knowledge base
    print("\n🧪 Testing the Knowledge Base:")
    test_knowledge_base(kb_path)
    
    print("\n📝 Next Steps:")
    print("1. The knowledge base has been created at:", kb_path)
    print("2. Tool implementations have been updated with validated parameters")
    print("3. Use the tool_kb_interface.py module in your agents to access the knowledge base")
    print("   - Import the ToolKnowledgeBase class from tool_kb_interface")
    print("   - Create an instance with kb = ToolKnowledgeBase()")
    print("   - Use kb.suggest_parameter_value(api_name, endpoint_name, param_name) to get suggested values")